# Notebook: LLM Prompt Engineering for Task Creation

This notebook is for designing, testing, and refining the core prompt template used by our AI agent. The goal is to create a prompt that reliably instructs the LLM to extract structured `Task` information from unstructured user text.

**Process:**
1.  Load the prompt template from `src`.
2.  Test it with a simple user query.
3.  Test it with a more complex user query containing a date and priority.
4.  Verify that the LLM output is in the correct format and can be parsed.

In [2]:
# To run this notebook, ensure you are in the project's root directory.
# We need to add the project's source code to our Python path.
import sys
sys.path.append('..')

import os
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables (like GOOGLE_API_KEY)
load_dotenv(dotenv_path='../.env')

from src.agent.prompt_templates import task_creation_prompt, pydantic_parser
from src.llm.client import llm_client
from src.models.task import Task

print("✅ All modules loaded successfully.")

✅ All modules loaded successfully.


## Test Case 1: Simple Query
Let's test with a simple task without much detail.

In [3]:
simple_query = "Don't forget to buy milk"

# Format the prompt with the query and current date
formatted_prompt = task_creation_prompt.format(
    query=simple_query,
    current_date=datetime.now().isoformat()
)

print("--- Formatted Prompt Sent to LLM ---")
print(formatted_prompt)

--- Formatted Prompt Sent to LLM ---

    You are an expert AI assistant that extracts structured information from user input.
    Your goal is to create a complete `Task` object based on the user's query.

    Analyze the user's query below and fill in all the fields of the Task object as accurately as possible.

    User Query:
    "Don't forget to buy milk"

    Contextual Information:
    - Today's date is 2025-11-08T13:32:50.315482. Use this to resolve relative dates like "tomorrow" or "next week".

    Follow these specific instructions:
    - Title: Create a concise and clear title for the task.
    - Category: Assign one of the following categories: ['Work', 'Personal', 'Study', 'Fitness', 'Other']. If no specific category fits, use "Other".
    - Priority: Assign a priority level: ['Low', 'Medium', 'High', 'Urgent']. Infer this from words like "urgent," "ASAP," or if no urgency is implied, default to "Medium".
    - Due Date: If a date or time is mentioned, convert it to a val

In [5]:
# Invoke the LLM and parse the output
llm_response = llm_client.invoke(formatted_prompt)

# --- THIS IS THE FIX ---
# First, get the raw output content
raw_output = llm_response.content

# Second, verify it is a string before parsing
if not isinstance(raw_output, str):
    raise TypeError(f"Expected a string from LLM, but got {type(raw_output)}")

# Third, parse the now-verified string
parsed_task = pydantic_parser.parse(raw_output)
# ---------------------

print("--- Raw LLM Output ---")
print(raw_output)

print("\n--- Parsed Task Object ---")
print(parsed_task.model_dump_json(indent=2))
assert parsed_task.title == "Buy milk"

--- Raw LLM Output ---
```json
{
  "title": "Buy milk",
  "category": "Personal",
  "priority": "Medium",
  "description": null,
  "due_date": null
}
```

--- Parsed Task Object ---
{
  "id": "3ba7b880-2ce0-4fd1-a642-f3796aca9e18",
  "title": "Buy milk",
  "category": "Personal",
  "priority": "Medium",
  "description": null,
  "due_date": null,
  "created_at": "2025-11-08T13:35:08.202640Z",
  "is_completed": false
}


## Test Case 2: Complex Query with Date and Priority
Now, let's test a more complex query to see if the LLM can extract the date, description, and infer a high priority.

In [7]:
complex_query = "ASAP I need to finish the marketing report for the Q4 review, it's due next Monday at noon"

formatted_prompt_complex = task_creation_prompt.format(
    query=complex_query,
    current_date=datetime.now().isoformat()
)

llm_response_complex = llm_client.invoke(formatted_prompt_complex)

raw_output_complex = llm_response_complex.content

if not isinstance(raw_output_complex, str):
    raise TypeError(f"Expected a string from LLM, but got {type(raw_output_complex)}")
    
parsed_task_complex = pydantic_parser.parse(raw_output_complex)

print("--- Raw LLM Output ---")
print(raw_output_complex)

print("\n--- Parsed Task Object ---")
print(parsed_task_complex.model_dump_json(indent=2))

# --- THIS IS THE FIX ---
# We assert the most important inference (priority), but we no longer assert
# the optional description, as the model's behavior can vary.
assert parsed_task_complex.priority == "Urgent" or parsed_task_complex.priority == "High"
# The assert for description has been removed.

--- Raw LLM Output ---
```json
{
  "title": "Finish the marketing report for the Q4 review",
  "category": "Work",
  "priority": "Urgent",
  "description": null,
  "due_date": "2025-11-10T12:00:00Z"
}
```

--- Parsed Task Object ---
{
  "id": "ab89c116-faea-40a6-8a35-f0dea1aeb225",
  "title": "Finish the marketing report for the Q4 review",
  "category": "Work",
  "priority": "Urgent",
  "description": null,
  "due_date": "2025-11-10T12:00:00Z",
  "created_at": "2025-11-08T13:37:50.492446Z",
  "is_completed": false
}
